In [21]:
import pandas as pd
import time
from bs4 import BeautifulSoup as bs
from selenium import webdriver
# 브라우저의 태그 검색 find.elements or find.element
from selenium.webdriver.common.by import By
# 키보드의 특수 이벤트 (ENTER,DELETE, CLEAR)
from selenium.webdriver.common.keys import Keys

In [22]:
url = 'https://kr.investing.com/economic-calendar'

In [23]:
# webdriver로 브라우져 열기
driver = webdriver.Chrome()

In [24]:
# investing 화면으로 이동
driver.get(url)

In [25]:
# 해당 웹 접속 시 로그인 팝업 닫기 버튼
len(driver.find_elements(By.CLASS_NAME, 'popupCloseIcon'))

1

In [26]:
driver.find_element(By.CLASS_NAME,'popupCloseIcon').click()

In [27]:
# 웹페이지 html 구조 파싱
soup = bs(driver.page_source,'html.parser')

In [28]:
# 테이블 위치로 축소
tb_data = soup.find('table',attrs={'id':'economicCalendarData'})

In [29]:
tb_body = tb_data.find('tbody')

In [30]:
tb_list =tb_body.find_all('td',attrs={'class':'event'})

In [31]:
len(tb_list)

54

In [32]:
href_list = []
for href in tb_list:
    try:
        data = href.find('a')['href']
        href_list.append(data)
    except:
        continue
    
href_list

['/economic-calendar/cleveland-cpi-887',
 '/economic-calendar/thomson-reuters-ipsos-pcsi-1879',
 '/economic-calendar/thomson-reuters-ipsos-pcsi-2068',
 '/economic-calendar/brazil-thomson-reuters-ipsos-pcsi-1882',
 "/economic-calendar/ecb's-lane-speaks-1971",
 '/economic-calendar/10-year-note-auction-571',
 '/economic-calendar/brazilian-foreign-exchange-flows-764',
 '/economic-calendar/federal-budget-balance-369',
 '/economic-calendar/ext.-migration---visitors-1449',
 '/economic-calendar/permanent-long-term-migration-1500',
 '/economic-calendar/visitor-arrivals-304',
 '/economic-calendar/foreign-bonds-buying-987',
 '/economic-calendar/foreign-investments-in-japanese-stocks-1451',
 '/economic-calendar/mi-inflation-expectations-365',
 '/economic-calendar/building-approvals-22',
 '/economic-calendar/private-house-approvals-1504',
 '/economic-calendar/rba-assistant-governor-jones-speaks-2138',
 '/economic-calendar/thai-consumer-confidence-803',
 '/economic-calendar/swiss-ppi-960',
 '/econom

In [33]:
base_url = 'https://kr.investing.com'

In [34]:
driver.get(base_url+href_list[0])

In [35]:
driver.page_source

'<html dir="ltr" xmlns="http://www.w3.org/1999/xhtml" xml:lang="ko" xmlns:schema="http://schema.org/" class="kr js csstransitions" lang="ko" geo="KR"><head><script async="" src="https://secure.cdn.fastclick.net/js/cnvr-coreid/latest/coreid.min.js"></script><script async="" src="https://sb.scorecardresearch.com/beacon.js"></script><script async="" src="https://secure.cdn.fastclick.net/js/cnvr-launcher/latest/launcher.min.js"></script><script async="" defer="" src="https://launchpad.privacymanager.io/latest/launchpad.bundle.js"></script><script src="https://cdn.hadronid.net/hadron.js?url=https%3A%2F%2Fkr.investing.com%2Feconomic-calendar%2Fcleveland-cpi-887&amp;ref=&amp;_it=amazon&amp;partner_id=715"></script><script type="text/javascript" async="" src="https://script.4dex.io/a/latest/adagio.js" id="adagiojs-292edb221bd146" data-pid="1306"></script>\n    \t    <script type="text/javascript" async="" src="//promos.investing.com/eu-w37htfhcq2/vendor/9a63d23b-49c1-4335-b698-e7f3ab10af6c/use

In [36]:
# 웹페이지 html 구조 파싱
soup2 = bs(driver.page_source,'html.parser')

In [39]:
#section 태그에서 id가 'leftColumn태그 추출
section_data = soup2.find(
    'section',
    attrs={
        'id' : 'leftColumn'
    }
)

In [42]:
file_name = section_data.h1.get_text().strip()

In [43]:
# driver에서 id가 eventTabDiv_history_0 태그 선택
element = driver.find_element(By.ID,'eventTabDiv_history_0')
# div 태그 선택
element.find_element(By.CLASS_NAME,'showMoreReplies').click()
# a 태그 선택

In [44]:
# 데이터를 더 불러오고 난 뒤 html 파싱
soup3 = bs(driver.page_source,'html.parser')

In [45]:
div_data = soup3.find('div',attrs={'id':'eventTabDiv_history_0'})

In [51]:
# div_data을 문자열로 변경 -> pandas에 내장된 read_html함수를 호출
df = pd.read_html(
    str(div_data)
)[0]

C:\Users\Owner\AppData\Local\Temp\ipykernel_19544\71939109.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(


In [53]:
# 데이터프레임을 파일로 저장
df.to_csv(f'{file_name[:10]}.csv',index=False)